<a href="https://colab.research.google.com/github/vilasha/ollama-sandbox/blob/master/Meeting_to_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert meeting recording (audio only) to minutes and actions

This Colab notebook implements an end-to-end pipeline for transcribing a meeting audio file and generating structured meeting minutes with action items using open-source models.

Key components:

1. Speech-to-Text Transcription
   Loads an English audio file (MP3) from Google Drive.
   Uses OpenAI's whisper-small.en via Hugging Face's pipeline("automatic-speech-recognition") in FP16 on GPU to produce a full transcript (timestamps available but unused).

2. Meeting Minutes Generation
   Prompts Meta's Llama-3.2-3B-Instruct (loaded with 4-bit quantization via BitsAndBytes for efficient GPU inference) to convert the raw transcript into structured markdown minutes.
   The prompt requests summary (including attendees, location, date), discussion points, takeaways, and action items with owners.
   Applies a chat template, uses a TextStreamer for live output, and renders the final response as Markdown.

Supporting setup:
   Mounts Google Drive, logs into Hugging Face (via stored token), installs/upgrades bitsandbytes and accelerate for quantization support.

The workflow is lightweight, runs on free Colab resources (enabled by 4-bit quantization of the 3B model), and combines a strong ASR model with a compact instruction-tuned LLM for summarization and action-item extraction.

In [3]:
!pip install -q --upgrade bitsandbytes accelerate
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

# Define models to call. For speech recognition we will be using WhisperSmall
# from OpenAI and for converting meeting minutes to action items Llama from Meta
WHISPER = "openai/whisper-small.en"
LLAMA = "meta-llama/Llama-3.2-3B-Instruct"

# Connect Google Drive, it becomes accessible like if it was mounted local disk
# On the first run of the Notebook there will be a pop-up message asking to
# allow to connect to the Google Drive
drive.mount("/content/drive")
# I saved the file at https://drive.google.com/file/d/1P86u__2pIJ9E8GAdw4RHFU0Ry_Y4LPZE/view?usp=sharing
# In my Google Drive it resides at /Colab Notebooks/src/council_010421_2022101V.mp3
# The file is from one of the HuggingFace datasets: https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main
audio_filename = "/content/drive/MyDrive/Colab Notebooks/src/council_010421_2022101V.mp3"

# Sign in to HuggingFace Hub
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

# Open the file
audio_file = open(audio_filename, "rb")

# Speech recognition
pipe = pipeline(
    "automatic-speech-recognition",
    model=WHISPER,
    dtype=torch.float16,
    device='cuda',
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print(transcription)

# Convert transcription to action items
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

# Quantify Llama model, so it works faster and takes less space on the local disk
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)
response = tokenizer.decode(outputs[0])
display(Markdown(response))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cuda


 So I think we are ready to go. Let's go ahead and start the meeting. Good afternoon, colleagues, the January 4 2021 meeting of the Seattle City Council will come to order it's two o'clock p.m. I'm learning that one's Alice president of the council will the clerk please call the role. Councilmember Strauss. Present. Councilmember her pulled here. Councilmember Juarez. here councilmember Lewis councilmember Morales here councilmember Peterson here councilmember Sawant here council president Gonzalez here seven present thank you madam clerk presentations I'm not aware of any presentations today so we'll move to approval of minutes the minutes of the city council meeting of to December 14th, 2020 have been reviewed. If there is no objection, the minutes will be signed. Hearing no objection, the minutes are being signed. Will the clerk please affix my signature to the minutes. Adoption of the referral calendar. If there's no objection, the January 4th, 2021 introduction and referral calend

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Jan 2026

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 So I think we are ready to go. Let's go ahead and start the meeting. Good afternoon, colleagues, the January 4 2021 meeting of the Seattle City Council will come to order it's two o'clock p.m. I'm learning that one's Alice president of the council will the clerk please call the role. Councilmember Strauss. Present. Councilmember her pulled here. Councilmember Juarez. here councilmember Lewis councilmember Morales here coun

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Jan 2026

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 So I think we are ready to go. Let's go ahead and start the meeting. Good afternoon, colleagues, the January 4 2021 meeting of the Seattle City Council will come to order it's two o'clock p.m. I'm learning that one's Alice president of the council will the clerk please call the role. Councilmember Strauss. Present. Councilmember her pulled here. Councilmember Juarez. here councilmember Lewis councilmember Morales here councilmember Peterson here councilmember Sawant here council president Gonzalez here seven present thank you madam clerk presentations I'm not aware of any presentations today so we'll move to approval of minutes the minutes of the city council meeting of to December 14th, 2020 have been reviewed. If there is no objection, the minutes will be signed. Hearing no objection, the minutes are being signed. Will the clerk please affix my signature to the minutes. Adoption of the referral calendar. If there's no objection, the January 4th, 2021 introduction and referral calendar will be adopted. Hearing no objections, the introduction and referral calendar is adopted. Approval of the agenda. If there is no objection, the agenda will be adopted. Hearing no objection, the agenda is adopted. Colleagues, welcome back. Happy New Year. This is the first full council meeting of 2021. Excited for us to kick off this meeting and looking forward to, as I said this morning, a year of lots of hard work before us this year and beyond. So at this time, we are gonna go ahead to open the remote public comment period for items on the city council agenda, introduction and referral calendar and the council's work program. I wanna thank everyone for their ongoing patience and cooperation as we continue to operate this remote public comment system. It does remain the strong intent of the city council to have remote public comment regularly included on our meeting agendas. However, as a reminder, the city council reserves the right to end or eliminate these public comment periods at any point if we deem that the system is being abused or is no longer suitable for allowing our meetings to be conducted efficiently and effectively. I'll moderate the public comment period in the following manner. The public comment period for this meeting is 20 minutes and each speaker will be given two minutes to speak. I'll call on each speaker by name and in the order in which they registered on the council's website. If you are listening and have not yet registered to speak but would like to, you can still sign up before the end of public comment by going to the council's website Seattle dot gov forward slash council COU and CIL. The public comment link is also listed on today's agenda. Once I call the speaker's name, staff will unmute the appropriate microphone and the speaker will hear an automatic prompt of you have been unmuted. After the speaker hears that automatic prompt, the speaker will need to press star six before speaking. Please begin speaking by stating your name and the item that you are addressing as a reminder public comment should relate to an item on today's agenda the introduction of referral calendar or the council support program. At about 10 seconds the speaker will hear a chime. That chime is your cue to begin wrapping up your public comment as you will only have 10 seconds of your allotted two minutes left to address the council. If speakers don't end their comments at the end of the allotted time provided the speaker's microphone will be muted to allow us to call on the next speaker. Once you've completed your public comments I would ask that you please disconnect from the line and if you plan to continue following this meeting you can do so via Seattle Channel or the listening options listed on the agenda. The public comment period is now open and we will begin with the first speaker on the list. Again as a reminder the speaker will hear the prompt you have been unmuted and then the speaker needs to press star 6 after the speaker hears a prompt if you have been unmuted. First up is Ariana Loriano followed by Howard Gale. Ariana, welcome. Hello, can you hear me? We can hear you. Hello, my name is Ariana Loriano. I'm an American with a disability and a renter in the university district in Seattle and I'm calling in in support of council member Sawant's bill to extend the eviction moratorium by one year. I'm an underpaid essential worker. I've worked the entirety of the pandemic. My expenses went up with COVID and my pay went down. I've lost wages to quarantine almost every month. Meanwhile, my burden has increased and the workplace has become deadly. Healthcare expenses also rose drastically in 2021 for me and many others to account for the minimum wage raise and now I owe $4,000 to an abusive slumlord. I've already cut into meds and foods at time to make rent, which has impacted my health and further risks my stability. My household has been living on ramen and rice for the past year, and my dog has gone hungry at times. We're formerly homeless, so we know what is coming. I have a tent and camping gear ready. I lived paycheck to paycheck before this pandemic, and this isn't getting better, it's getting worse. Without extending the moratorium or finding a way to erase this debt, we will be homeless again. I have nowhere to cut from except for food, medicine, and healthcare, and I am in very poor health. I know in my heart that my life and the lives of many others rest in your hands upon your policies. My name is Ariana Lloriano, and if I die on the streets of Seattle in 2021, it will be your political choice. You may have saved us from COVID only to kill us with poverty. Thank you for calling in and sharing with us your experience. Next up is Howard Gale, and I do have Tana Yasu and Bob Andrew listed, but showing as not present. So, Tana and Bob, if you are listening, now is the time to call back in to the list in line so that we have an opportunity to call on you. Next up, Howard Gale. Howard, welcome. Hi. Hi. Good afternoon. Happy New Year. Howard Gale, District 7, commenting on police accountability. Today you're going to be voting on a law establishing and clarifying subpoena powers of the Office of Police Accountability and the Office of the Inspector General. Contingent upon the police unions accepting this law during the current contract bargaining. This highlights two well-established failures of our so-called police accountability system. One, that justice and community safety can and has been routinely given away by the city when bargaining union contracts with police. And second, that the city council's predilection perform over substance when it comes to police accountability. Unless the right questions are asked in a police use of force investigation, unless an officer's claims are subject to critical inquiry, then tools like subpoenas are completely meaningless. For example, in the OPA investigation into the 2019 SPD murder of Ryan Smith, officers made claims concerning their fears that Ryan Smith's girlfriend was seriously injured and facing imminent harm, despite the abundant evidence that 911 operators knew for a fact that not to be the case. Yet in the OPA investigation, no 911 personnel or SPD command staff were interviewed to determine how this happened or if the officers were lying. subpoenas are irrelevant without truly independent investigations. Seattle needs true police accountability, as other U.S. cities voted to have this last November, based on real community oversight with an elected body capable of investigating and disappointing police and, of course, having subpoena powers. But subpoenas in the hands of people that are not asking the right questions is not gonna give us accountability. Thanks. Thank you, Howard, for calling in. Okay, one last call for Tana Yasu or Bob Andrew. Still seen on the pre registration list that they are not present. Just gonna give it a couple of seconds here to see if either of them call in. I'm going to turn it over to the public comment and begin addressing items of business on our agenda. First up is payment of the bills. We do have three bills to be a little tedious, but I appreciate your patience here. So I will ask that the clerk please read the bill titles into the record. Council Bill 119983 through 119985, a property mind to pay a start-up of claims to the week of December 7th, 2020 through December 11th, 2020 and ordering payment thereof. A property mind to pay a start-up of claims to the week of December 14th, 2020 through December 18th, 2020 and ordering the payment thereof. And a property of mind to pay certain claims for the week of December 21st, 2020 through December 24th, 2020 and ordering the payment thereof. Thank you, Madam Clerk. I move to pass council bill 119983. Is there a second? Second. Thank you. It's been moved and seconded that council bill 119983 pass. Are there any comments? Hearing no comments, will the clerk please call the roll on the passage of council bill 119983. Drowse. Yes. Herbold. Yes. Juarez. Aye. Lewis. Yes. Morales. Yes. Peterson. Yes. Sawant. Yes. President Gonzalez. Aye. Eight in favor, none opposed. Thank you, Madam Clerk. I will now move to pass Council Bill 119984. Is there a second? Second. Thank you. It's been moved and seconded that Council Bill 119984 pass. Are there any comments? Hearing no comments, will the clerk please call the roll on the passage of Council Bill 119984? Strauss. Yes. Herbold. Yes. Juarez. Lewis. Yes. Thank you Madam Clerk, I am now going to move to pass Council Bill 119-985 is there a second? Council President Gonzalez, excuse me. I'm aware Madam Clerk, I'm going to circle back. Thank you. Yep. Okay, it's been moved and seconded that Council Bill 119-985 pass. Are there any comments? Hearing it, no comments. Will the clerk please call the roll on the passage of Council Bill 119-985. Strauss. Yes. Terpold. Yes. Juarez. Aye. Lewis. Yes. Morales. Yes. Peterson. Yes. Sawant. Yes. Thank you Madam Clerk, all three bills pass and the chair will sign all three council bills. Will the clerk please affix my signature to this legislation on my behalf. Thank you so much. We're going to go ahead and move into committee reports. Will the clerk please read agenda item one into the record. item one council bill 11997 for relating to civilian and community oversight of the police creating a subpoena process for the office of police accountability and office of inspector general for public safety while ensuring due process for individuals who are the subject of this pina and adding new sections 3.29.126 and.1245 of the sale on the city code the committee recommends the bill pass as amended thank you madam clerk councilmember herbal you are chair of this committee and are recognized in order to provide the committee's report. Thank you so much. Really appreciate that. And appreciate that the ordinance title here is so complete. Doesn't leave me a whole lot to say. But as I mentioned in council briefings this morning, this new legislation adds sections to the municipal code and codifies and affirms the city's stance that the Office of Police Accountability and the Inspector General can seek subpoenas of those who may have witnessed or been involved in potential misconduct incidents. The legislation creates a process for the OPA and the OIG to directly issue the subpoena and enforce them by seeking a court order should the subject of the subpoena fail to comply. The new legislation specifically requires that individuals and third-party record holders serve the subpoena are provided a written notice of their right to do process. This specific protection was not codified previously and this addition is intended to increase civilian participation in OPA investigations and OIG audits and reviews. subpoena authority for the Seattle Police Officers Guild and the Seattle Police Management association is still subject to collective bargaining requirements and the notice that is referenced in the the bill itself recognizes as well that search warrant requirements need to be separately met for certain types of evidence. And I just want to mention and thank the efforts of the office of police accountability, the office of the inspector general, the city the attorney's office, the mayor's office, and the ACLU and the community police commission in their work in helping to develop this legislation. The 2017 accountability legislation sponsored by Council President Gonzalez did include subpoena authority and this bill builds on that legislation in order to provide clarity on the procedures and due process protections. Thank you. Thank you, Council Member Herbold for that report. Are there any additional comments on the bill? Okay, not hearing any additional comments, I would just say thank you Council Member Herbold and also thank you to Mayor Durkin for her partnership with with you and with many of the other accountability entities to advance this additional clarity when we initially considered the police accountability ordinance in 2017, we knew that there needed to be details that would need to be sorted out in order to effectuate the intent behind the subpoena power. But I think that, um, and I think that the details included in this legislation accomplish that original intent and really do, um, uh, implement much of what needs to be implemented in order to add additional transparency and more teeth to what we had envisioned in that part of the ordinance. So thank you for all of your hard work and thanks for all of the folks who have contributed to developing the legislation in the past several months to allow us an opportunity to consider this bill and I do intend to support it and really am appreciative of your efforts here. All right, hearing and seeing no other comments, we're going to go ahead and close out a debate, and I would ask that the clerk please call the roll on the passage of the bill. Strauss. Yes. Herbold. Yes. Juarez. Aye. Lewis. Yes. Morales. Yes. Peterson. Yes. Sawant. Yes. President Gonzalez. Aye. Thank you Madam Clerk, the bill passes and the chair will sign up. Will the clerk please affix my signature to the legislation on my behalf. Okay, moving along now to the report of the transportation and utilities committee. Will the clerk please read the short title of item 2 into the record. The report, the transportation and utilities committee agenda item 2, council 119976 relating to the east marginal way grade separation project, authorizing director of the Seattle department of transportation to acquire, accept and record on behalf of the city of Seattle three quick claim deeds and an assignment, an assumption of easement agreement for the east marginal way overpass from the port of Seattle. The committee recommends it will pass. Thank you, Madam Clerk. Even the short title of that bill is rather impressive, so I'm going to go ahead and recognize Councilmember Peterson who is the chair of the committee so that he may provide the report of the committee. Councilmember Peterson. Thank you Council President. Colleagues on today's agenda we've got 13 items from the Transportation Utilities Committee and this includes four council bills and nine mayoral appointments to the transportation related advisory boards. I discussed these items during our council briefing this morning. This first item council bill 119976 authorizes SDOT Seattle Public Utilities and Seattle City Light to accept easements from the port of Seattle regarding a completed overpass at East marginal way just south of Spokane Street. This council bill was recommended unanimously by the transportation and utilities committee. Thank you. Thank you so much, Council Member Peterson. Are there any additional comments on the bill? Here are no additional comments on the bill. Will the clerk please call the roll on the passage of the bill. Drowse. Yes. Herbold. Yes. Juarez. Aye. Lewis. Yes. Morales. Yes. Peterson. Yes. Sawant. Yes. President Gonzalez. Aye. Eight in favor, none opposed. The bill passes and the chair will sign up. Will the clerk please affix my signature to the legislation on my behalf. Item number three, will the clerk please read the short title of item three into the record. Agenda item three, Council Bill 119955 relating to Seattle Public Utilities relating to certain properties of the city of Renton at the intersection of Interstate 405 and Seattle Public Utilities Cedar River pipelines right away. Committee recommends that bill pass. Thank you, Madam Clerk. Councilmember Peterson, floor is yours. Thank you, Council President. Council Bill 119955 swaps small pieces of land near the city of Renton between the Seattle City, between Seattle Public Utilities and the Washington State Department of Transportation to accommodate freshwater pipeline that exists there. This bill also was recommended unanimously by the Transportation Committee. Thank you, Councilmember Peterson. Are there any additional comments on the bill? Hearing no additional comments on the bill, will the clerk please call the roll on the passage of the bill? Strauss. Yes. Herbold. Yes. Juarez. Aye. Lewis. Yes. Morales. Yes. Peterson. Yes. Sawant. Yes. President Gonzalez. Aye. Eight in favor, none opposed. The bill passes and the chair will sign up. Will the clerk please affix my signature to the legislation on my behalf. Item number four, will the clerk please read the short title of item four into the record. Agenda item four, Council 1, 1, 9, 9, 5, 8, relating to the Cedar River Watershed authorizing two years of ecological thinning in accordance with the Cedar River Watershed habitat conservation plan. The committee recommends the bill pass. Thank you, Madam Clerk. Council Member Peterson, you are recognized in order to provide the committee's report. Thank you, Council President, Council Bill 119958 authorizes additional time to complete the necessary ecological thinning of trees to improve forest habitat and biodiversity. In accordance with the Cedar River Watershed Habitat Conservation Plan, both the Muckleshoot Tribe and the Sierra Club supported this two-year extension with letters to the Council. And this was also unanimously approved by the Transportation and Utilities Committee. Thank you Councilmember Peterson for those comments. Are there any additional comments on the bill? Hearing no additional comments on the bill, will the clerk please call the roll on the passage of the bill? Drowse? Yes. Herbold? Yes. Juarez? Aye. Lewis? Yes. Morales? Yes. Peterson? Yes. Sawat? Yes. President Gonzalez? Aye. Eight in favor, none opposed. Thank you, Madam Clerk. The bill passes and the chair will sign it. Will the clerk please affix my signature to the legislation on my behalf. Item number five. Will the clerk please read the short title of item five into the record. Agenda item five, Council Bill 119963 relating to Seattle Public Utilities updating water regulations to conform to current standards making technical corrections. The committee recommends the bill pass. Thank you, Madam Clerk, Council Member Peterson, this one is also yours, you're recognized in order to provide the committee's report. Thank you, Council President, Council Bill 119963 is the title, explains updates the water regulations, we actually heard this bill over two of our committee meetings to give extra time for consideration, the agenda item contains a PowerPoint presentation outlining the nine changes that are being made, the committee recommended this unanimously as well. Thank you. much. Councilmember Peterson, are there any additional comments on the bill? Hearing no additional comments on the bill, will the clerk please call the roll on the passage of the bill? Strouse? Yes. Purple? Yes. Juarez? Aye. Lewis? Yes. Morales? Peterson? Yes. Sawa? Yes. President Gonzalez? Aye. 8 in favor, none opposed. Thank you so much. The bill passes and the chair will sign it. Will the clerk please affix my signature to the legislation on my behalf. Items 6 through 9. Will the clerk please read items 6 through 9 into the record. Item 6 through 9, appointment 1711 through 1714. Appointment of Yasir al-Fargas, member of Seattle Bicycle Advisory Board for term to August 31, 2021 appointment every appointments of my Mona Raheem Andrew L Dennenberg and Sarah Olden-Hoffin as member Seattle Bicycle Advisory Board for term to August 31st 2022. The committee recommends these appointments be confirmed. Thank you madam clerk councilmember Peterson you are recognized to provide the committee's report. Thank you council president. Colleagues the council president and our city clerk's office have been very helpful in organizing these appointments today. We're going We're going to vote on nine appointments, but in just three motions because we're grouping them together based on the advisory board. So the bicycle advisory board is one vote that we just read into the record, and then the transit advisory board, and then the pedestrian advisory board. And I'll speak briefly now to all nine appointments, even though we'll take three separate votes. Colleagues, as you know, each mode of transportation has its own separate master plan, so we've got separate advisory boards. Over Zoom, members of a transportation committee on December 16 were able to meet and ask questions of all the new appointees. I believe these applicants came across as qualified and deeply interested in these important transportation issues. I believe they also indicated open-minded and balanced views on transportation in order to help us break down the different silos of each mode and see the bigger picture of of an interconnected transportation network that can keep us mobile and protect our environment. The committee unanimously recommended all of these appointments, both new appointments and reappointments. Thank you. Thank you, council member Peterson. Are there any additional comments on the appointments to the Seattle Bicycle Advisory Board? Hearing no additional comments on these appointments, will the clerk please call the roll on the confirmation of the appointments. Strauss. Yes. Purple. Yes. Juarez. Aye. Lewis. Yes. Morales. Yes. Peterson. Yes. Sawant. Yes. President Gonzalez. Aye. Eight in favor, none opposed. The motion carries. Any appointments are confirmed. Agenda items 10 through 12. Will the clerk please read items 10 through 12 to the record. Agenda items 10 through 12. Appointments 15 through 1717. Appointment and reappointments of author Ken Yuki, Haiko Budesh, and Barbara A. Wright as member of Seattle Transit Advisory Board for term to August 2, 2022. The committee recommends these appointments be confirmed. Thank you, Madam Clerk, Councilmember Peterson. Anything more to add? Yes, Council President. Just for the Transit Transit Advisory Board, they're even more important now that voters renew this funding for the Seattle Transportation Benefit District. The Transit Advisory Board is an official advisor on the STBD. So we look forward to working closely with this advisory board over the coming years as we implement STBD and as transit ridership grows as the economy recovers. you. Thank you. Councilmember Peterson, are there any additional comments on these appointments? Hearing no additional comments on these appointments, will the clerk please call the roll on the confirmation of the appointments? Strauss? Yes. Herbold? Yes. Juarez? Aye. Lewis? the motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. The motion carries. Lee Ghandi as members of the pedestrian advisory board for term to March 31st, 2021. And appointment Emily D. Davis as members of the pedestrian advisory board for term to March 31st, 2022. The committee recommends these appointments be confirmed. Thank you, Madam Clerk. Councillor Peterson, you're recognized in order to provide the committee report on these appointments as well. Thank you, council president. The pedestrian advisory board, as we're opening up new sound transit stations, the pedestrian advisory board is very important we have a lot of people walking to those stations now. And so we're going to be looking for a lot of coordination among the advisory boards as we try to connect our network and as transit ridership grows. Thank you. Thank you, Councilmember Peterson. Are there any additional comments on these appointments? Hearing no additional comments, will the Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. Yes. We will move to adjournment. This does conclude the items of business on today's agenda. Our next regularly scheduled city council meeting is on Monday, January 11, 2021 at 2 o'clock p.m. I hope that you all have a wonderful afternoon. We're adjourned, colleagues. Thank you very much.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Meeting Minutes**
=====================

**Summary**
------------

* Date: January 4, 2021
* Location: Seattle City Council Chambers
* Attendees: Council Members Strauss, Herbold, Juarez, Lewis, Morales, Peterson, Sawant, and President Gonzalez
* Agenda Items:
	+ Council Bill 119983
	+ Council Bill 119984
	+ Council Bill 119985
	+ Council Bill 119976
	+ Council Bill 119958
	+ Appointments to various advisory boards

**Discussion Points**
--------------------

* Introduction and referral calendar
* Approval of agenda
* Public comment period
* Council Bill 119983 (payment of bills)
* Council Bill 119984 (payment of bills)
* Council Bill 119985 (payment of bills)
* Council Bill 119976 (east marginal way grade separation project)
* Council Bill 119958 (Cedar River Watershed habitat conservation plan)
* Council Bill 119963 (Seattle Public Utilities updating water regulations)
* Appointments to Seattle Bicycle Advisory Board
* Appointments to Seattle Transit Advisory Board
* Appointments to Pedestrian Advisory Board

**Takeaways**
-------------

* Council Bills 119983, 119984, and 119985 passed unanimously
* Council Bill 119976 passed unanimously
* Council Bill 119958 passed unanimously
* Council Bill 119963 passed unanimously
* Appointments to various advisory boards confirmed

**Action Items**
-----------------

* None

**Owners**
------------

* Council Bill 119983: Council President Gonzalez
* Council Bill 119984: Council President Gonzalez
* Council Bill 119985: Council President Gonzalez
* Council Bill 119976: Council President Gonzalez
* Council Bill 119958: Council President Gonzalez
* Council Bill 119963: Council President Gonzalez
* Appointments to Seattle Bicycle Advisory Board: Council President Gonzalez
* Appointments to Seattle Transit Advisory Board: Council President Gonzalez
* Appointments to Pedestrian Advisory Board: Council President Gonzalez<|eot_id|>